In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

from torchvision import transforms, datasets
import torchvision.transforms.functional as fn
import torchvision.transforms
import torchmetrics
from PIL import Image

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
import wandb
WANDB_LOGGING = True
CONFIG = {
    "project_name": "viktor_test_runs",
    "dataloader": {
        "batch_size": 96
    },
    "bias": True,
    "lr": 0.0001
}
SWEEP_CONFIG = {
    'method': 'random',
    'name': 'sweep',
    'metric': {
        'goal': 'minimize', 
        'name': 'validation_loss'
        },
    'parameters': {
        'batch_size': {'values': [16, 32, 64]},
        'epochs': {'values': [5, 10, 15]},
        'lr': {'max': 0.1, 'min': 0.0001}
    }
}

## Loading and Transforming the Dataset

In this project we are working with images of vegetables and classifying them into 15 distinct classes.

The images were downloaded from: https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset. There are 21,000 images in total. The images are already split into 3 sets - training(70%), validation(15%) and testing(15%). However, the percentages (provided on the website) are only rough estimations - in actuality, there are 15,000 training images, and 3000 of each - validation and testing images - which makes it a 71,4285...%, 14,2857...%, 14,2857...% split...

We begin by loading the images from the downloaded repository using the `ImageFolder` object from the `torchvision.datasets` module, immediately giving them transformations defined in the two following blocks of code. First of all, we need to make sure that the images are the same size, so we resize all of them to 224x224 pixels. In addition to that, we augment the training images by doing random rotations and random resized crops (within a range of 80-100% of the original image). 

In [ ]:
transform = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
])
train_dataset = ImageFolder('data/Vegetable Images/train', transform=transform)
dataloader = DataLoader(train_dataset, batch_size=CONFIG["dataloader"]["batch_size"], shuffle=True)

In [ ]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

mean, std = get_mean_and_std(dataloader)
mean, std

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
train_dataset = ImageFolder('data/Vegetable Images/train', transform=train_transform)
val_dataset = ImageFolder('data/Vegetable Images/validation', transform=train_transform)
test_dataset = ImageFolder('data/Vegetable Images/test', transform=train_transform)

train_dataloader = DataLoader(train_dataset, batch_size=CONFIG["dataloader"]["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=CONFIG["dataloader"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=CONFIG["dataloader"]["batch_size"], shuffle=False)

print(f"Number of Training Images: {len(train_dataset)} - Validation Images: {len(val_dataset)} - Test Images: {len(test_dataset)}")
print(f"Number of target classes: {len(train_dataset.classes)}")
print(f"Names of the classes: {(train_dataset.classes)}")

An example of a training image after augmentation

In [ ]:
images, labels = next(iter(train_dataloader))
fn.to_pil_image(images[0])

## Defining the Neural Network Architecture

In [ ]:
torch.cuda.is_available()

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_linear_classifier(input_shape: int, output_shape: int, hidden_dim: int = 32):
    return nn.Sequential(
        nn.Linear(in_features=input_shape, out_features=hidden_dim),
        nn.ReLU(),
        nn.Linear(in_features=hidden_dim, out_features=hidden_dim),
        nn.ReLU(),
        nn.Linear(in_features=hidden_dim, out_features=hidden_dim),
        nn.ReLU(),
        nn.Linear(in_features=hidden_dim, out_features=output_shape),
        nn.Softmax()
    )

In [ ]:
network_1 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size= (7,7), stride=(2,2), padding=(3,3), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),

    nn.Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(128),

    nn.Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.Conv2d(256, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(256),
)

In [ ]:
network_2 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size= (7,7), stride=(2,2), padding=(3,3), bias=CONFIG['bias']),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    
    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    
    nn.Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.ReLU(),
    nn.BatchNorm2d(128),
)

In [ ]:
network_2_a = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size= (7,7), stride=(2,2), padding=(3,3), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    
    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    
    nn.Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(128),
    nn.ReLU(),
)

In [ ]:
network_3 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size= (7,7), stride=(2,2), padding=(3,3), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),

    nn.Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(128),

    nn.Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.Conv2d(256, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(256),

    nn.Conv2d(256, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(512),
    nn.ReLU(),
    nn.Conv2d(512, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(512),
)

In [ ]:
network_4 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size= (7,7), stride=(2,2), padding=(3,3), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),

    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),

    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),

    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
)

In [ ]:
network_5 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size= (7,7), stride=(2,2), padding=(3,3), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),

    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),

    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),

    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
)

In [ ]:
network_6 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size= (7,7), stride=(2,2), padding=(3,3), bias=CONFIG['bias']),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=CONFIG['bias']),
    nn.BatchNorm2d(64),
)

In [ ]:
class VegetableClassifier(nn.Module):
    def __init__(self, loss_fn: callable, network: nn.Sequential, network_out: int, num_classes: int=15) -> None:
        super(VegetableClassifier, self).__init__()
        self.loss_fn = loss_fn
        self.classifier_out = num_classes
        self.device = DEVICE
        self.network = network
        self.avg_maxpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.linear_classifier = get_linear_classifier(network_out, self.classifier_out)
        
    def forward(self, input):
        input = self.network(input)
        input = self.avg_maxpool(input)
        input = torch.flatten(input, 1)
        return self.linear_classifier(input)
    
    def train_step(self, X, y):
        X, y = X.to(self.device), y.to(self.device)
        y_pred = self(X)
        return self.loss_fn(y_pred, y)
    
    def val_step(self, X, y, accuracy):
        X, y = X.to(self.device), y.to(self.device)
        val_pred = self(X)
        return self.loss_fn(val_pred, y), accuracy(val_pred, y)
                

In [ ]:
from tqdm.auto import tqdm  # We use tqdm to display a simple progress bar, allowing us to observe the learning progression.

def get_accuracy(pred, y):
    pred = pred.argmax(dim=1) # Get the largest value for each image - the predicted class
    correct = torch.eq(y, pred).sum().item()
    acc = (correct / len(pred)) 
    return acc

def fit(
  model: nn.Module,
  num_epochs: int,
  optimizer: torch.optim.Optimizer,
  train_dataloader: DataLoader,
  val_dataloader: DataLoader,
  print_rate: int = 100
  ):
    accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=model.classifier_out, average="weighted").to(model.device)
    model = model.to(model.device)
    # Iterate through epochs with tqdm
    for epoch in tqdm(range(num_epochs)):
        print(f"Epoch: {epoch}\n")
        train_loss = 0
        model.train()  # Set mode of model to train
        
        for batch, (X, y) in enumerate(train_dataloader):
            loss = model.train_step(X, y)
            train_loss += loss.item()
            
            # Getting the loss gradient and making an optimizer step
            optimizer.zero_grad()  
            loss.backward()
            optimizer.step()

            if batch % print_rate == 0: 
                print(f"Looked at {batch} Batches\t---\t{batch * len(X)}/{len(train_dataloader.dataset)} Samples")
            elif batch == len(train_dataloader) - 1:
                print(f"Looked at {batch} Batches\t---\t{len(train_dataloader.dataset)}/{len(train_dataloader.dataset)} Samples")
        
        # Divide the train_loss by the number of batches to get the average train_loss
        avg_train_loss = train_loss / len(train_dataloader)

        # Validation
        # Setup the Val Loss and Accuracy to accumulate over the batches in the val dataset
        val_loss = 0
        val_acc = 0
        # Set model to evaluation mode and use torch.inference_mode to remove unnecessary training operations 
        model.eval()
        with torch.inference_mode():
            for X_val, y_val in val_dataloader:
                loss, acc = model.val_step(X_val, y_val, accuracy)
                val_loss += loss.item()
                val_acc += acc

        # Get the average Val Loss and Accuracy
        avg_val_loss = val_loss / len(val_dataloader)
        avg_val_acc = val_acc / len(val_dataloader)

        print(f"Train loss: {avg_train_loss} | Val Loss: {avg_val_loss} | Val Accuracy: {avg_val_acc}")
        if WANDB_LOGGING:
            wandb.log({"Train Loss": avg_train_loss,"Val Loss": avg_val_loss, "Val Accuracy": avg_val_acc})

In [ ]:
def train():
    loss_fn = nn.CrossEntropyLoss()
    model = VegetableClassifier(loss_fn=loss_fn, network=network_2, network_out=128)
    print(f"Total parameters: {sum(param.numel() for param in model.parameters())}")
    print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
    optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['lr'], betas=(0.9, 0.99))
    fit(model=model, num_epochs=10, optimizer=optimizer, train_dataloader=train_dataloader, val_dataloader=val_dataloader)
    return model

In [ ]:
wandb.init(project=CONFIG['project_name'])
model = train()
wandb.finish()

In [ ]:
torch.save(model.state_dict(), "./models/network_2_lr_0001.pt")